### Задача идентификации взломщика по его поведению в сети Интернет

Ссылка: [Catch Me If You Can](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

У нас есть данные по посещениям пользователями каких-то сайтов и времени посещения.

Необходимо определить сессии в тесте, которые осуществил определенный юзер. Его класс в трейне 1, все остальные юзеры 0.

В ноутбуке приведен алгоритм создания спарс матрицы из сайтов, которые посещали люди из выборки. В каждой строке будет от 1 до 10 непустых элементов.

По времени никаких фич не построено, это для самостоятельной работы.

In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score as auc
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import lil_matrix
from sklearn.model_selection import KFold, GridSearchCV
import time


%matplotlib inline
pd.set_option('display.max_columns', None)

In [183]:
data = pd.read_csv('./data/Alice/train_sessions.csv')
test = pd.read_csv('./data/Alice/test_sessions.csv')

In [184]:
test.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [185]:
data.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [186]:
#data.info()

In [187]:
num_cols = [col for col in data.columns if data[col].dtype=='int64' or
                                           data[col].dtype=='float64']
num_cols.remove('target')
time_cols = [col for col in data.columns if data[col].dtype=='object']

In [188]:
for col in time_cols:
    data[col] = pd.to_datetime(data[col], yearfirst=True)
    test[col] = pd.to_datetime(test[col], yearfirst=True)

In [189]:
for col in time_cols:
   # data[col + 'year'] = data[col].dt.year;
   # test[col + 'year'] = test[col].dt.year;
   # data[col + 'month'] = data[col].dt.month;
   # test[col + 'month'] = test[col].dt.month;
   # data[col + 'day'] = data[col].dt.day;
   # test[col + 'day'] = test[col].dt.day;
    data[col + 'hours'] = data[col].dt.hour;
    test[col + 'hours'] = test[col].dt.hour;
   # data[col + 'minuts'] = data[col].dt.minute;
   # test[col + 'minuts'] = test[col].dt.minute;
   # data[col + 'seconds'] = data[col].dt.second;
   # test[col + 'seconds'] = test[col].dt.second;
    data.drop(col, axis=1, inplace=True)
    test.drop(col, axis=1, inplace=True)
    

In [190]:
#test.info()

In [191]:
num_cols = [col for col in data.columns if data[col].dtype=='int64' or
                                           data[col].dtype=='float64']

test_num_cols = num_cols.copy()
test_num_cols.remove('target')
                     
#Заполним отсутствующие сайты уникальным значением.
data[num_cols] = data[num_cols].fillna(-1)
test[test_num_cols] = test[test_num_cols].fillna(-1)
#Это необходимо для того, чтобы данные по сайтам привести к целочисленному типу.
data[num_cols] = data[num_cols].astype(int)
test[test_num_cols] = test[test_num_cols].astype(int)

In [192]:
#test.info()

In [193]:
data.head()

,session_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target,time1hours,time2hours,time3hours,time4hours,time5hours,time6hours,time7hours,time8hours,time9hours,time10hours
0,1,718,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,10,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2,890,941,3847,941,942,3846,3847,3846,1516,1518,0,11,11,11,11,11,11,11,11,11,11
2,3,14769,39,14768,14769,37,39,14768,14768,14768,14768,0,16,16,16,16,16,16,16,16,16,16
3,4,782,782,782,782,782,782,782,782,782,782,0,10,10,10,10,10,10,10,10,10,10
4,5,22,177,175,178,177,178,175,177,177,178,0,10,10,10,10,10,10,10,10,10,10


In [194]:
test.head()

,session_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,time1hours,time2hours,time3hours,time4hours,time5hours,time6hours,time7hours,time8hours,time9hours,time10hours
0,1,29,35,22,321,23,2211,6730,21,44582,15336,11,11,11,11,11,11,11,11,11,11
1,2,782,782,782,782,782,782,782,782,782,782,11,11,11,11,11,11,11,11,11,11
2,3,55,55,55,55,55,55,55,55,1445,1445,15,15,15,15,15,15,15,15,15,15
3,4,1023,1022,50,222,202,3374,50,48,48,3374,10,10,10,10,10,10,10,10,10,10
4,5,301,301,301,66,67,69,70,68,71,167,15,15,15,15,15,15,15,15,15,15


In [200]:
from sklearn.metrics import roc_auc_score, accuracy_score

cv = KFold(n_splits=5, shuffle=True, random_state=777)
rfc = RandomForestClassifier(n_estimators=15, random_state=777, n_jobs=-1)
params = {'max_features': [4, 7, 10, 13, 17, 20], 'max_depth': [4, 6, 9]}
gs = GridSearchCV(rfc, param_grid=params, cv=cv, scoring='roc_auc')
gs.fit(data[test_num_cols],  data['target'])
rfc_pred = gs.best_estimator_.predict_proba(test[test_num_cols])

array([[9.99363243e-01, 6.36756945e-04],
       [9.99379737e-01, 6.20263334e-04],
       [9.99883111e-01, 1.16888800e-04],
       ...,
       [9.99918372e-01, 8.16275261e-05],
       [9.99918372e-01, 8.16275261e-05],
       [9.99929130e-01, 7.08698031e-05]])

In [173]:
gs.best_score_

0.985868338205671

In [201]:
rfc_pred[:, 1]

array([6.36756945e-04, 6.20263334e-04, 1.16888800e-04, ...,
       8.16275261e-05, 8.16275261e-05, 7.08698031e-05])

In [202]:
#пишем функцию для сабмита и делаем сабмит

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [203]:
write_to_submission_file(rfc_pred[:, 1], 'submission.csv')